# Modèle pour la détermination du type de véhicule

### 1. Préparation des données

Transformation : Niveau de gris sur tout le dataset

In [31]:
import cv2, os

output_folder = './gray_images'

for image_name in os.listdir('./result') :
    img = cv2.imread(f'./result/{image_name}')
    grayFrame = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    output_path = os.path.join(output_folder, image_name)
    cv2.imwrite(output_path, grayFrame)


Transformation : Floutage des images de meilleure qualité

Transformation : UpSampling en retournant les images aléatoirement

In [ ]:
import tensorflow as tf
import numpy as np

def random_rotate_image(opencv_img):
    img = tf.convert_to_tensor(opencv_img, dtype=tf.float32)
    img = tf.image.rot90(img)
    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_flip_up_down(img)

    img_rotated = img.numpy().astype(np.uint8)
    return img_rotated

output_folder = './rotated_gray_images/'
os.makedirs(output_folder, exist_ok=True)

for image_name in os.listdir('./gray_images') :
    img = cv2.imread(f'./gray_images/{image_name}')

    rotations_list = [random_rotate_image(img), random_rotate_image(img)]
    for i, rotated_image in enumerate(rotations_list) :
        img_name = os.path.splitext(image_name)[0]
        category = img_name.split("_")[0]
        output_image_name = f"{img_name}_{i}.png"
        output_path = os.path.join(output_folder + f'{category}/', output_image_name)
        cv2.imwrite(output_path, rotated_image)

Importation des datasets

In [44]:
import os
import tensorflow as tf

# Définition des chemins pour les fichiers image
train_dir = "./rotated_gray_images"
val_dir = ""
test_dir = ""

# Définition des constantes 
batch_size = 32
img_height = 224  # Hauteur de l'image (modifiable selon vos besoins)
img_width = 224 

# # Charger les données d'entraînement depuis le répertoire
# x_train = tf.keras.utils.image_dataset_from_directory(
#     train_dir,
#     image_size=(img_height, img_width),
#     batch_size=batch_size,
#     label_mode=None 
# )

# file_paths = x_train.file_paths

# # Extraire les labels depuis les noms de fichiers
# image_paths = [os.path.basename(path) for path in file_paths]
# y_train = [path.split('_')[0] for path in image_paths] 

# print(y_train[:10])

train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='int',
    color_mode='grayscale'
)

Found 1528 files belonging to 2 classes.


Création du modèle

In [45]:
from tensorflow.keras import layers, models

model = tf.keras.Sequential([
        layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(img_width, img_height, 1)),
        layers.MaxPooling2D(),
        layers.Flatten(),
        layers.Dense(32, activation='relu'),
        layers.Dense(len(train_dataset.class_names), activation='softmax')
    ])

model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

Training the Model

In [46]:
history = model.fit(
    train_dataset,
    epochs=5,
    verbose=2
)

Epoch 1/5
48/48 - 32s - loss: 173.9916 - accuracy: 0.5596 - 32s/epoch - 667ms/step
Epoch 2/5
48/48 - 37s - loss: 0.6873 - accuracy: 0.6126 - 37s/epoch - 766ms/step
Epoch 3/5
48/48 - 39s - loss: 0.6838 - accuracy: 0.6126 - 39s/epoch - 813ms/step
Epoch 4/5
48/48 - 39s - loss: 0.6805 - accuracy: 0.6126 - 39s/epoch - 816ms/step
Epoch 5/5
48/48 - 39s - loss: 0.6776 - accuracy: 0.6126 - 39s/epoch - 815ms/step
